# Travel Agent Planner with AgentNeo Integration

This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with a Travel Agent Planner system. AgentNeo provides seamless tracing capabilities for both function calls and AI model interactions, allowing for comprehensive analysis and debugging of complex systems.


# Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [1]:
import os
import requests
from dotenv import load_dotenv
from litellm import completion
import openai
from openai import OpenAI

In [2]:
# Load environment variables
load_dotenv("enter your .env file path ")


# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")



In [3]:
# Initialize AgentNeo Package
import os
os.chdir('..')

from agentneo import AgentNeo, Tracer, Evaluation,launch_dashboard
# Initialize AgentNeo session
# Create project
neo_session = AgentNeo(session_name="ai_travel_agent_session1")

project_name = "ai_travel_agent_demo1"

try:
    neo_session.create_project(project_name=project_name)
    print("Project created successfully")
except:
    neo_session.connect_project(project_name=project_name)
    print("Project connected successfully")
# Start tracing
tracer = Tracer(session=neo_session)
tracer.start()

Project connected successfully
Project 'ai_travel_agent_demo1' found.
Tracing Started.


# Travel Agent Tools
Now, let's define our Travel Agent Planner Tools with AgentNeo integration.

In [4]:
@tracer.trace_llm(name="llm_call")
def llm_call(prompt, max_tokens=512, model="gpt-4o-mini"):
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0.7,
    )

    return response.choices[0].message.content.strip()


# Tools outside agents
@tracer.trace_tool(name="weather_tool")
def weather_tool(destination):

    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    params = {"q": destination, "appid": api_key, "units": "metric"}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]

        return f"{weather_description.capitalize()}, {temperature:.1f}°C"
    except requests.RequestException:
        return "Weather data not available."


@tracer.trace_tool(name="currency_converter_tool")
def currency_converter_tool(amount, from_currency, to_currency):
    api_key = os.environ.get("EXCHANGERATE_API_KEY")
    base_url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"

    try:
        response = requests.get(base_url)
        response.raise_for_status()
        data = response.json()

        if data["result"] == "success":
            rate = data["conversion_rate"]
            return amount * rate
        else:
            return None
    except requests.RequestException:
        return None


@tracer.trace_tool(name="flight_price_estimator_tool")
def flight_price_estimator_tool(origin, destination):
    # This is a mock function. In a real scenario, you'd integrate with a flight API.
    api_key = os.environ.get("FLIGHT_API_KEY")
    # Implement actual API call here
    return f"Estimated price from {origin} to {destination}: $500-$1000"


# Agent with persona
@tracer.trace_agent(name="itinerary_agent")
class ItineraryAgent:
    def __init__(self, persona="Itinerary Agent"):
        self.persona = persona

    def plan_itinerary(self, user_preferences, duration=3):
        itinerary_prompt = f"""
You are a travel expert named {self.persona}.
Based on the following user preferences, create a {duration}-day travel itinerary.

User Preferences:
{user_preferences}

Itinerary:
"""
        return llm_call(itinerary_prompt, max_tokens=512)





# Running the Analysis
Now let's create a  main function for our Travel Agent Planner which recall all tools 

In [5]:
# Main function

@tracer.trace_agent(name="travel_agent")
def travel_agent():
    print("Welcome to the Personalized Travel Planner!\n")

    # Get user input
    user_input = input("Please describe your ideal vacation: ")

    # Extract preferences
    preferences_prompt = f"""
Extract key travel preferences from the following user input:
"{user_input}"

Please provide the extracted information in this format:
Destination:
Activities:
Budget:
Duration (in days):
"""
    extracted_preferences = llm_call(preferences_prompt)
    print("\nExtracted Preferences:")
    print(extracted_preferences)

    # Parse extracted preferences
    preferences = {}
    for line in extracted_preferences.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            preferences[key.strip()] = value.strip()

    # Validate extracted preferences
    required_keys = ["Destination", "Activities", "Budget", "Duration (in days)"]
    if not all(key in preferences for key in required_keys):
        print("\nCould not extract all required preferences. Please try again.")
        return

    # Fetch additional information
    weather = weather_tool(preferences["Destination"])
    print(f"\nWeather in {preferences['Destination']}: {weather}")

    origin = input("Please enter your departure city: ")
    flight_price = flight_price_estimator_tool(origin, preferences["Destination"])
    print(flight_price)

    # Plan itinerary
    itinerary_agent = ItineraryAgent()
    itinerary = itinerary_agent.plan_itinerary(
        extracted_preferences, int(preferences["Duration (in days)"])
    )
    print("\nPlanned Itinerary:")
    print(itinerary)

    # Currency conversion
    budget_amount = float(preferences["Budget"].replace("$", "").replace(",", ""))
    converted_budget = currency_converter_tool(budget_amount, "USD", "INR")
    if converted_budget:
        print(f"\nBudget in INR: {converted_budget:.2f} INR")
    else:
        print("\nCurrency conversion not available.")

    # Generate travel summary
    summary_prompt = f"""
Summarize the following travel plan:

Destination: {preferences['Destination']}
Activities: {preferences['Activities']}
Budget: {preferences['Budget']}
Duration: {preferences['Duration (in days)']} days
Itinerary: {itinerary}
Weather: {weather}
Flight Price: {flight_price}

Travel Summary:
"""
    travel_summary = llm_call(summary_prompt, max_tokens=2048)
    print("\nTravel Summary:")
    print(travel_summary)

In [6]:
# Main function to run the travel agent
def main():
    travel_agent()

# Ensure the script runs only when executed directly
if __name__ == "__main__":
    main()
    tracer.stop()


Welcome to the Personalized Travel Planner!



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Extracted Preferences:
Destination: Karela  
Activities: Nature  
Budget: $100  
Duration (in days): 10

Weather in Karela: Weather data not available.
Estimated price from banglore to Karela: $500-$1000


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Planned Itinerary:
### 10-Day Itinerary for Kerala, India (Nature-Focused)

#### Overview
Kerala, known as "God's Own Country," offers stunning landscapes, lush greenery, and diverse wildlife. This itinerary balances natural beauty with budget-friendly activities, perfect for nature enthusiasts.

#### Day 1: Arrival in Kochi
- **Morning:** Arrive at Cochin International Airport. Take a local bus to Fort Kochi (approx. $2).
- **Afternoon:** Explore Fort Kochi’s beaches and the iconic Chinese fishing nets.
- **Evening:** Stroll through the historic streets, visit St. Francis Church, and enjoy street food (approx. $5).

#### Day 2: Munnar
- **Morning:** Take a local bus to Munnar (approx. $5). 
- **Afternoon:** Visit the Tea Museum (entry fee approx. $1) and learn about Kerala's tea production.
- **Evening:** Explore local markets and enjoy traditional Kerala cuisine at a local eatery (approx. $7).

#### Day 3: Munnar - Nature Exploration
- **Morning:** Trek to Anamudi Peak (free). Enjoy

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 38
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 37



Travel Summary:
**Travel Summary for Kerala, India**

- **Destination:** Kerala, India
- **Focus:** Nature activities
- **Budget:** $100 (excluding flight)
- **Duration:** 10 days

**Overview:**
Kerala, known as "God's Own Country," is celebrated for its stunning landscapes, lush greenery, and diverse wildlife. This 10-day itinerary is designed for nature lovers on a budget.

**Itinerary Highlights:**

- **Day 1:** Arrive in Kochi, explore Fort Kochi's beaches and historic sites, and enjoy street food.
- **Day 2:** Travel to Munnar, visit the Tea Museum, and savor local cuisine.
- **Day 3:** Trek to Anamudi Peak, visit Eravikulam National Park, and relax at a café.
- **Day 4:** Head to Thekkady, explore Periyar Wildlife Sanctuary, and enjoy a cultural program.
- **Day 5:** Participate in a spice plantation tour, birdwatching, and dine on traditional Kerala meals.
- **Day 6:** Travel to Alleppey, rent a canoe to explore the backwaters.

**Additional Notes:**
- Local transportation and 

# Metrics Evaluation
Supported Metrics
Goal Decomposition Efficiency (goal_decomposition_efficiency)
Goal Fulfillment Rate (goal_fulfillment_rate)
Tool Call Correctness Rate (tool_call_correctness_rate)
Tool Call Success Rate (tool_call_success_rate)

In [8]:
exe = Evaluation(session=neo_session, trace_id=tracer.trace_id)

In [ ]:
# run a single metric
exe.evaluate(metric_list=['goal_decomposition_efficiency', 
                         'goal_fulfillment_rate', 
                         'tool_call_correctness_rate', 
                         'tool_call_success_rate'])

In [14]:
#print metric result
metric_results = exe.get_results()
metric_results

[{'metric_name': 'goal_fulfillment_rate',
  'score': 0.4,
  'reason': "The system responses provide a detailed itinerary for a 10-day trip to Kerala, focusing on nature-related activities, which aligns with the user's intent. However, the budget of $100 is not realistically achievable given the estimated flight costs of $500 to $1000 from Bangalore to Kerala, which are not included in the budget. Additionally, while the itinerary includes various affordable activities and dining options, the overall cost of transportation and activities may exceed the user's budget when considering the total expenses for 10 days. The lack of weather data and currency conversion tools also limits the comprehensiveness of the travel guide. Therefore, while the itinerary is well-structured and relevant, the significant discrepancy between the user's budget and the estimated flight costs leads to a lower fulfillment rate.",
  'result_detail': {'metric_name': 'goal_fulfillment_rate',
   'config': {},
   're

In [9]:
neo_session.launch_dashboard()

INFO:root:Port 3000 is busy. Finding an available port...
INFO:root:Using port 3002
INFO:root:Dashboard launched successfully. Access it at: http://localhost:3002
